In [1]:
# %pip install numpy
# %pip install pandas
# %pip install torch

In [2]:
#installing the required libraries
# %pip install transformers
# %pip install torchtext
# %pip install sentencepiece
# %pip install datasets
# %pip install torchmetrics
# %pip install matplotlib
# %pip install seaborn
# %pip install scikit-learn
# #huggiface download
# %pip install huggingface_hub


In [3]:
# %pip install llama-cpp-python

In [4]:
from huggingface_hub import hf_hub_download

/home/biomedialab/anaconda3/envs/viveksdmlenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from llama_cpp import Llama

In [6]:
#download the model llama 2 13b chat gguf
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGUF"
model_basename = "llama-2-13b-chat.Q5_K_S.gguf" # the model is in gguf format

In [7]:
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

#print toal path
print(model_path)

/home/biomedialab/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGUF/snapshots/4458acc949de0a9914c3eab623904d4fe999050a/llama-2-13b-chat.Q5_K_S.gguf


In [8]:
# GPU
lcpp_llm = None
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, # CPU cores
    n_batch=512, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=32 # Change this value based on your model and your GPU VRAM pool.
    )
     

llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from /home/biomedialab/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGUF/snapshots/4458acc949de0a9914c3eab623904d4fe999050a/llama-2-13b-chat.Q5_K_S.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 5120
llama_model_loader: - kv   4:                          llama.block_count u32              = 40
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 13824
llama_model_loader: - kv   6:                 llama.rop

In [9]:
prompt= f"List the factors of 50?"

prompt_template=f'''You are a math assistant. I will ask you to find factors of given number. Please answer in the correct format. For example, if I ask 'List the factors of 15?' , you should answer 'factors=[1, 3, 5, 15]'. Do not include any other information in your answer.
USER: {prompt}

ASSISTANT:'''
        

In [10]:
response=lcpp_llm(prompt=prompt_template, max_tokens=500, temperature=0.00001, top_p=0.95,
                  repeat_penalty=1.2, top_k=150,
                  echo=False)


llama_print_timings:        load time =    4636.77 ms
llama_print_timings:      sample time =       6.15 ms /    15 runs   (    0.41 ms per token,  2439.02 tokens per second)
llama_print_timings: prompt eval time =    4636.61 ms /    88 tokens (   52.69 ms per token,    18.98 tokens per second)
llama_print_timings:        eval time =    9516.08 ms /    14 runs   (  679.72 ms per token,     1.47 tokens per second)
llama_print_timings:       total time =   14193.16 ms /   102 tokens


In [11]:
print(response["choices"][0]["text"])

 factors=[1, 2, 4, 50]


In [12]:
import json

In [13]:
#load file in data folder int_addition.json
with open('primality_test.json') as f:
    data = json.load(f)

In [14]:
#data as pandas dataframe
import pandas as pd
df = pd.DataFrame(data)
#last 5 rows
df

,primes,composites
0,3187,99160
1,59509,12005
2,58787,33309
3,22861,73881
4,89519,78681
...,...,...
4995,38053,83445
4996,74797,35077
4997,78823,18386
4998,61751,19887


In [16]:
sorted_df = pd.DataFrame({'primes': df.sort_values('primes')['primes'].values, 'composites': df.sort_values('composites')['composites'].values})

sorted_df

,primes,composites
0,2,33
1,23,34
2,31,45
3,37,82
4,41,126
...,...,...
4995,99881,99938
4996,99901,99941
4997,99907,99954
4998,99929,99964


In [17]:
import time


for i in range(0, len(df)):
    
    if i%100==0:
        print(f"Completed {i} rows")
        
    try:
        
        prompt= f"List the factors of {sorted_df.iloc[i, 1]}?"
        
        prompt_template=f'''You are a math assistant. I will ask you to find factors of given number. Please answer in the correct format. For example, if I ask 'List the factors of 15?' , you should answer 'factors=[1, 3, 5, 15]'. Do not include any other information in your answer.

        USER: {prompt}
        
        ASSISTANT:'''
        
        response=lcpp_llm(prompt=prompt_template, max_tokens=500, temperature=0.00001, top_p=0.95,
                    repeat_penalty=1.2, top_k=150,
                    echo=False)
        
        with open('composite_factors.txt', 'a') as f:
            answer=response["choices"][0]["text"]
            f.write(f"{sorted_df.iloc[i, 1]} = {answer} \n")
            
            
    except:
        
        with open('composite_factors.txt', 'a') as f:
            f.write(f"{sorted_df.iloc[i, 1]} = Error \n")
            
        time.sleep(2)

Completed 0 rows


Llama.generate: prefix-match hit

llama_print_timings:        load time =    4636.77 ms
llama_print_timings:      sample time =       6.52 ms /    16 runs   (    0.41 ms per token,  2455.87 tokens per second)
llama_print_timings: prompt eval time =    1210.72 ms /    22 tokens (   55.03 ms per token,    18.17 tokens per second)
llama_print_timings:        eval time =   10179.27 ms /    15 runs   (  678.62 ms per token,     1.47 tokens per second)
llama_print_timings:       total time =   11433.41 ms /    37 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =    4636.77 ms
llama_print_timings:      sample time =       8.00 ms /    17 runs   (    0.47 ms per token,  2125.00 tokens per second)
llama_print_timings: prompt eval time =     625.80 ms /    11 tokens (   56.89 ms per token,    17.58 tokens per second)
llama_print_timings:        eval time =   10792.41 ms /    16 runs   (  674.53 ms per token,     1.48 tokens per second)
llama_print_timings:       to

Completed 100 rows



llama_print_timings:        load time =    4636.77 ms
llama_print_timings:      sample time =      10.79 ms /    26 runs   (    0.42 ms per token,  2409.64 tokens per second)
llama_print_timings: prompt eval time =     646.36 ms /    11 tokens (   58.76 ms per token,    17.02 tokens per second)
llama_print_timings:        eval time =   16886.53 ms /    25 runs   (  675.46 ms per token,     1.48 tokens per second)
llama_print_timings:       total time =   17609.93 ms /    36 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =    4636.77 ms
llama_print_timings:      sample time =       9.14 ms /    22 runs   (    0.42 ms per token,  2408.06 tokens per second)
llama_print_timings: prompt eval time =     661.07 ms /    12 tokens (   55.09 ms per token,    18.15 tokens per second)
llama_print_timings:        eval time =   14137.70 ms /    21 runs   (  673.22 ms per token,     1.49 tokens per second)
llama_print_timings:       total time =   14865.24 ms /    33 

Completed 200 rows


...


llama_print_timings:        load time =    4636.77 ms
llama_print_timings:      sample time =      13.37 ms /    32 runs   (    0.42 ms per token,  2394.13 tokens per second)
llama_print_timings: prompt eval time =     772.49 ms /    13 tokens (   59.42 ms per token,    16.83 tokens per second)
llama_print_timings:        eval time =   21543.33 ms /    31 runs   (  694.95 ms per token,     1.44 tokens per second)
llama_print_timings:       total time =   22415.69 ms /    44 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =    4636.77 ms
llama_print_timings:      sample time =      13.79 ms /    32 runs   (    0.43 ms per token,  2320.52 tokens per second)
llama_print_timings: prompt eval time =     652.13 ms /    12 tokens (   54.34 ms per token,    18.40 tokens per second)
llama_print_timings:        eval time =   21249.04 ms /    31 runs   (  685.45 ms per token,     1.46 tokens per second)
llama_print_timings:       total time =   22003.92 ms /    43 

Completed 3700 rows



llama_print_timings:        load time =    4636.77 ms
llama_print_timings:      sample time =      12.41 ms /    30 runs   (    0.41 ms per token,  2417.60 tokens per second)
llama_print_timings: prompt eval time =     655.32 ms /    12 tokens (   54.61 ms per token,    18.31 tokens per second)
llama_print_timings:        eval time =   19646.25 ms /    29 runs   (  677.46 ms per token,     1.48 tokens per second)
llama_print_timings:       total time =   20392.84 ms /    41 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =    4636.77 ms
llama_print_timings:      sample time =       9.62 ms /    23 runs   (    0.42 ms per token,  2391.85 tokens per second)
llama_print_timings: prompt eval time =     682.12 ms /    12 tokens (   56.84 ms per token,    17.59 tokens per second)
llama_print_timings:        eval time =   14916.60 ms /    22 runs   (  678.03 ms per token,     1.47 tokens per second)
llama_print_timings:       total time =   15668.78 ms /    34 

Completed 3800 rows



llama_print_timings:        load time =    4636.77 ms
llama_print_timings:      sample time =      13.27 ms /    31 runs   (    0.43 ms per token,  2336.80 tokens per second)
llama_print_timings: prompt eval time =     786.09 ms /    12 tokens (   65.51 ms per token,    15.27 tokens per second)
llama_print_timings:        eval time =   20897.93 ms /    30 runs   (  696.60 ms per token,     1.44 tokens per second)
llama_print_timings:       total time =   21779.32 ms /    42 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =    4636.77 ms
llama_print_timings:      sample time =      14.93 ms /    36 runs   (    0.41 ms per token,  2411.74 tokens per second)
llama_print_timings: prompt eval time =     669.40 ms /    12 tokens (   55.78 ms per token,    17.93 tokens per second)
llama_print_timings:        eval time =   23784.93 ms /    35 runs   (  679.57 ms per token,     1.47 tokens per second)
llama_print_timings:       total time =   24563.88 ms /    47 